In [7]:
import pandas as pd
import numpy as np
import math
from math import log
import seaborn as sns
from scipy.stats import linregress
import matplotlib.pyplot as plt

In [8]:
df1 = pd.read_csv('datasets/meteorite_landings.csv', on_bad_lines='warn')
df2 = pd.read_csv('datasets/meteorite_landings_on_earth.csv', on_bad_lines='warn')   #lendo os datasets
df3 = pd.read_csv('datasets/cneos_fireball_data.csv', on_bad_lines='warn')

# Pouso de meteoritos no planeta terra

- Iara Araujo
- Beatriz Barreto 
- Paloma Duarte